# Analysis of Immigration Canada Subreddit

## Data Cleaning

In [3]:
import pandas as pd
import datetime as dt
from datetime import datetime
import seaborn as sns
from ydata_profiling import ProfileReport
from wordcloud import WordCloud
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt


### Data Import

In [4]:
# Importing reddit data from csv
df_all = pd.read_csv('reddit_all_posts.csv')
df_mixed = pd.read_csv('reddit_mixed_posts.csv')
df_top = pd.read_csv('reddit_top_posts.csv')

### Duplicates

### Missing submission body

### Missing category & Other category

In [1]:
# Wordcloud application

### Unsupported media